In [0]:
#dbutils.widgets.text("TABLE_INPUT_1", "g3_catalog.bronze.spark_sql_input_1")
#dbutils.widgets.text("TABLE_INPUT_2", "g3_catalog.bronze.spark_sql_input_2")
#dbutils.widgets.text("TABLE_INPUT_3", "g3_catalog.bronze.spark_sql_input_3")

In [0]:
TABLE_INPUT_1 = dbutils.widgets.get("TABLE_INPUT_1")
TABLE_INPUT_2 = dbutils.widgets.get("TABLE_INPUT_2")
TABLE_INPUT_3 = dbutils.widgets.get("TABLE_INPUT_3")

In [0]:
spark.sql(f"""
  CREATE OR REPLACE TABLE {TABLE_INPUT_1} AS
  SELECT
    claim_id,
    user_id,
    -- Convierte el BIGINT a DATE
    CAST(CAST(date / 1000000000 AS TIMESTAMP) AS DATE) AS date,
    diagnosis_code,
    amount,
    current_timestamp() AS inserted_at
  FROM read_files(
    'abfss://datalake@stdemdsai.dfs.core.windows.net/raw/airflow/G3/claims.parquet',
    format        => 'parquet',
    -- El schema de lectura sigue siendo BIGINT, que es el tipo real en el archivo
    schema        => 'claim_id STRING,
                      user_id STRING,
                      date BIGINT,
                      diagnosis_code STRING,
                      amount BIGINT'
  )
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"""
  CREATE OR REPLACE TABLE {TABLE_INPUT_2} AS
  SELECT
    user_id,
    patient_name,
    -- Conversión para birth_date
    CAST(CAST(birth_date / 1000000000 AS TIMESTAMP) AS DATE) AS birth_date,
    gender,
    -- Conversión para enrollment_date
    CAST(CAST(enrollment_date / 1000000000 AS TIMESTAMP) AS DATE) AS enrollment_date,
    -- plan_type,
    status,
    current_timestamp() AS inserted_at
  FROM read_files(
    'abfss://datalake@stdemdsai.dfs.core.windows.net/raw/airflow/G3/patients.parquet',
    format        => 'parquet',
    -- El schema de lectura define los tipos de datos del archivo original
    schema        => 'user_id STRING,
                      patient_name STRING,
                      birth_date BIGINT,
                      gender STRING,
                      enrollment_date BIGINT,
                      status STRING'
  )
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"""
  CREATE OR REPLACE TABLE {TABLE_INPUT_3} AS
  SELECT *, current_timestamp() AS inserted_at
  FROM read_files(
    'abfss://datalake@stdemdsai.dfs.core.windows.net/raw/airflow/G3/cie10_catalog.parquet',
    format        => 'parquet',
    -- El schema de lectura define los tipos de datos del archivo original
    schema        => 'diagnosis_code STRING,
                      description_es STRING'
  )
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"""
  DELETE FROM {TABLE_INPUT_1}
  WHERE claim_id IS NULL
""")

spark.sql(f"""
  DELETE FROM {TABLE_INPUT_2}
  WHERE user_id IS NULL
""")

spark.sql(f"""
  DELETE FROM {TABLE_INPUT_3}
  WHERE diagnosis_code IS NULL
""")

print("Se eliminaron las filas con id nulo.")

Se eliminaron las filas con id nulo.
